In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv


colors = [    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [9]:
# LOAD h5 file:
filename = '/home/cat/Downloads/sleap_video.mp4.analysis.h5'

hf = h5py.File(filename, 'r')

keys = hf.keys()
group2 = hf.get('tracks')
tracks = []
for k in range(len(group2)):
    tracks.append(group2[k])
    
tracks = np.array(tracks).squeeze().transpose(2,1,0)
print (tracks.shape)


[n_frames, n_animals, n-features, xy,] (7500, 4, 14, 2)


In [62]:
# LOAD TRACKLETS + IDS
# filename = '/home/cat/Downloads/sleap_video.mp4.tracker_flow.analysis.h5'

# hf = h5py.File(filename, 'r')

# keys = hf.keys()
# print ("keys; ", keys)
# group2 = hf.get('tracks')
# tracks_ids = []
# print (hf.get('track_names')[0])

# for k in range(3000):
#     print (np.unique(hf.get('track_occupancy')[k]))


# #for k in range(len(group2)):
# #    print (k, group2[k].shape)
# #     tracks_ids.append(group2[k])
    
# # tracks_ids = np.array(tracks_ids).squeeze()
# # print (tracks_ids.shape)

In [85]:
# MAKE COMPARISON OF LABELED VS> UNLABELED VIDEOS
# start going through video and 
video_name = '/home/cat/Downloads/sleap_video.mp4'
original_vid = cv2.VideoCapture(video_name)


# video sizes
size_vid = np.array([1280,1024])
dot_size = 16
    
#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = (video_name[:-4]+'_sleap_labels.mp4')
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)
    
# go through first videos
from tqdm import trange

start = 0
end = start+100

original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

for n in trange(start, end, 1):
    ret, frame = original_vid.read()

    for i in range(tracks.shape[1]):
 
        x = tracks[n,i,0]
        y = tracks[n,i,1]
        
        if np.isnan(x) or np.isnan(y):
            continue
            
        x = int(x)
        y = int(y)
        

        
        frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            #matplotlib.colors.to_rgb(colors[i]))*255.).astype('uint8')
            matplotlib.colors.to_rgb('white'))*255.).astype('uint8')

    video_out.write(frame)
      

video_out.release()
original_vid.release()
    

100%|██████████| 100/100 [00:01<00:00, 81.38it/s]


In [6]:
traces_inferences = []
ctr=0
start = 0
end = 89989

for n in trange(start, end, 1):
    #ind = n
    #print (n, frame_names[n])
    #print (data)
    # load inference locations
    traces_inferences.append([])
    dets = convertdetectiondict2listoflist(data[frame_names[n]], bpts)
    for i, det in enumerate(dets):
        traces_inferences[ctr].append([])
        for x, y, p, _ in det:
            traces_inferences[ctr][i].append([x,y])
     
    ctr+=1
traces_inferences = np.array(traces_inferences)
print (traces_inferences.shape)

np.save('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/traces_inferences.npy', traces_inferences)

100%|██████████| 89989/89989 [00:11<00:00, 7526.85it/s] 

(89989, 14)


In [2]:
traces_inferences = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/traces_inferences.npy', allow_pickle=True)
print (traces_inferences.shape)

In [5]:
# import assembled animal output
fname_csv_assembled = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
data_assembled = load_csv(fname_csv_assembled)[0]
data_assembled = np.array(data_assembled)
print(" Data assembled shape: ", data_assembled.shape)

 Data assembled shape:  (56, 89989, 3)


(89989, 14)
(56, 3)


In [ ]:
# LOAD VIDE TO ANNOTATE
video_name = "/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi"
frame_rate = 25


In [ ]:
# video writing step
#video_out.release()
#out.release()
#original_vid.release()
#cv2.destroyAllWindows()

# Robust outlier detection;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

       
original_vid = cv2.VideoCapture(video_name)
#cap = cv2.VideoCapture('chaplin.mp4')

ctr=0
# figure out index of starting frame and move video to location
frame_no1 = 0 #56*60*frame_rate
original_vid.set(1,frame_no1)

# initialize video writer for file to be saved
out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/vids_out/'


# Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
n_sec = 180
clrs = ['blue','red','yellow','green']

# 
#for k in range(tracesx.shape[1]):    
#for k in range(tracesx.shape[1])[:500]:    
ctr=0
start = 7*60*25
end = start + 120*25

fname_out = out_dir+'video_labeled_'+str(start)+"_"+str(end)+'.avi'
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
# fourcc = cv2.VideoWriter_fourcc(*'MP4V')
# fourCc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (1280,1024), True)


start_frame_number = start
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)

for k in range(start,end,1):    
    if k %100==0:
        print (k)
    
    #while True:
    ret, frame = original_vid.read()
    if ret==False:
        break

#         if ctr==k:
#             ctr+=1
#             break
#         ctr+=1
            
    #frame_cropped = np.flipud(frame.copy())#[:,150:1174][::2,::2]
    frame_cropped = frame.copy()  #[:,150:1174][::2,::2]

    # PLOT EVERY FEATURE
    threshold = 0.95
    if False:
        for p in range(tracesx.shape[0]):
            l = likelihoods[p,k]
            if l < threshold:
                continue

            x = np.int32(tracesx[p,k])

            y = np.int32(tracesy[p,k])


            # plot individual features
            frame_cropped[y-5:y+5,x-5:x+5]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p//14]))*255.).astype('uint8')
    
    # PLOT MEAN FEATURE
    for p in range(0, tracesx.shape[0], 14):
        l = likelihoods[p:p+14,k]
        idx = np.where(l<threshold)[0]

        traces_local = tracesx[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 1.)
        x = np.int32(np.nanmedian(traces_local))
        
        traces_local = tracesy[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 1.)
        y = np.int32(np.nanmedian(traces_local))
        
        # plot large bump
        frame_cropped[y-15:y+15,x-15:x+15]= (np.float32(
            matplotlib.colors.to_rgb(clrs[p//14]))*255.).astype('uint8')
    

    #cv2.imshow('frame',frame_cropped)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #   break

    video_out.write(frame_cropped)
    #out.write(frame_cropped)
    
video_out.release()
original_vid.release()
cv2.destroyAllWindows()

#break
    

In [11]:
fname = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/cnn_training/4.npy'
data = np.load(fname)
print (data.shape)
np.save(fname[:-4]+"44.npy", 
        data[:,
             data.shape[1]//2-100:data.shape[1]//2+100,
             data.shape[2]//2-100:data.shape[2]//2+100]
       )

plt.imshow(data[0])
plt.show()

(5000, 500, 500, 3)


In [10]:
data3 = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/cnn_training/11.npy')
print (data3.shape)
plt.imshow(data3[0])
plt.show()

(5000, 200, 200, 3)


In [5]:
fig=plt.figure()
data1 = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/0/0.npz')
images = data1['images']
print (images.shape)
plt.imshow(images[0])
plt.show()

(172, 200, 200, 3)
